In [1]:
import numpy as np
import string
from nltk import word_tokenize
import onnxruntime as nxrun

In [2]:
def preprocess(text):
   tokens = word_tokenize(text)
   # split into lower-case word tokens, in numpy array with shape of (seq, 1)
   words = np.asarray([w.lower() for w in tokens]).reshape(-1, 1)
   # split words into chars, in numpy array with shape of (seq, 1, 1, 16)
   chars = [[c for c in t][:16] for t in tokens]
   chars = [cs+['']*(16-len(cs)) for cs in chars]
   chars = np.asarray(chars).reshape(-1, 1, 1, 16)
   return words, chars

In [3]:
sess = nxrun.InferenceSession("./bidaf.onnx")

In [4]:
def answer(context, query):

    cw, cc = preprocess(context)
    qw, qc = preprocess(query)
    
    answer = sess.run(None, 
                  {'context_word': cw,
                   'context_char': cc,
                   'query_word': qw,
                   'query_char': qc})
    
    answer = sess.run(None, 
                  {'context_word': cw,
                   'context_char': cc,
                   'query_word': qw,
                   'query_char': qc})
    
    # assuming answer contains the np arrays for start_pos/end_pos
    start = np.asscalar(answer[0])
    end = np.asscalar(answer[1])
    return (" ".join([w for w in cw[start:end+1].reshape(-1)]))

In [5]:
context = 'Revel is a full-service digital consultancy that partners \
with organizations to lead go-to-market transformation by \
building seamless, simple, connected experiences. \
 \
We are problem solvers, designers and builders \
united around a core set of values and a commitment \
to solving complex challenges swiftly, efficiently and elegantly.'

In [6]:
query = 'What is Revel?'
answer(context, query)

'a full-service digital consultancy'

In [7]:
query = 'What does Revel do?'
answer(context, query)

'partners with organizations to lead go-to-market transformation by building seamless , simple , connected experiences'

In [8]:
query = 'How does Revel solve challenges?'
answer(context, query)

'swiftly , efficiently and elegantly'

In [9]:
query = 'Tell me about revel'
answer(context, query)

'we are problem solvers , designers and builders united around a core set of values'

In [10]:
query = 'Which is the best consultancy in the world?'
answer(context, query)

'revel'